Import Data

In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
import json
from pandas_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from gensim.parsing.preprocessing import remove_stopwords
import string

# Get database connection credentials
with open(os.path.join(os.getcwd(),'fb_marketplace_conn.json'), mode='r') as f:
    database_dict = json.load(f)
# Password for the database
RDS_pass = 'aicore2022!'
# Create engine to connect to database
engine = create_engine(f"{database_dict['DATABASE_TYPE']}+{database_dict['DBAPI']}://{database_dict['USER']}:{RDS_pass}@{database_dict['HOST']}:{database_dict['PORT']}/{database_dict['DATABASE']}")
# Import the tables as DataFrame
products = pd.read_sql('products', engine)
images = pd.read_sql('images', engine)

Profile data

In [ ]:
# https://www.analyticsvidhya.com/blog/2021/06/generate-reports-using-pandas-profiling-deploy-using-streamlit/products.info(verbose=True)
products.head()
prof = ProfileReport(products)
prof.to_file(output_file='products.html')

images.info(verbose=True)
images.head()
prof = ProfileReport(images)
prof.to_file(output_file='images.html')

# Pandas profiling outcome
    # # Product name 94.7 unique and mean lenght is 76 characters
    # # Category - 11.6% N/A and 436 distinact categories 
    # # product_desciprtion 935 N/A, mostly unique
    # # price 935 N/A  (so it seems N?As are acroos the board, but might not be) price is string
    # # url not missing - interesting as this might help get product name

In [ ]:

products_numeric_cols = products.select_dtypes(include=['number']).columns
print(products_numeric_cols)

products_non_numeric_cols = products.select_dtypes(exclude=['number']).columns
print(products_non_numeric_cols)

count number of products

In [ ]:
# Count number of products
print(f'The number of products in the database is {len(products)}')
# Are all the products actually unique?
unique_products = products['id'].nunique()
print(f'The number of unique products is {unique_products}')
# Count number of images
print(f'The number of images in the database is {len(images)}')
# Are all the images actually unique?
unique_images = images['id'].nunique()
print(f'The number of unique images is {unique_images}')
# Count unique create_date to see if it is all the same
create_time_unique = products['create_time'].nunique()
print(f'The number of unique create_times is {create_time_unique}')

Clean dataset

In [ ]:
# Good site https://www.justintodata.com/data-cleaning-techniques-python-guide/
# Get simpliest form to name a product
    # Extract just product name from product name
    # Could get the second from last index from url - seems the most complete version of the description - Could take hirearchy approach , name, url, description

# plt.figure(figsize=(10,8))
# Heat map to show missing data across colunms
# It shows that N/A values are across specific colunmns and that by removing these we would not lose any useful information
# colours = ['#000099', '#ffff00'] # specify colours: yellow - missing. blue - not missing
# sns.heatmap(products == 'N/A', cmap=sns.color_palette(colours))
# Remove N/A rows
# This method removes the rows completely from products, even when assigning to var
# products_na_removed = products.drop(products[products['category'] == 'N/A'].index, inplace=True)
# This methods filters products and can assign to the variable so that the products full dataframe remains
products_na_removed = products.loc[products['category'] != 'N/A']

# Make location useful
# Split by , to get the city and county
# Consider using this for mapping
# Remove the location field once split
products_na_removed[['city_town', 'county']] = products_na_removed['location'].str.split(',', expand=True)
products_na_removed.drop('location', axis=1, inplace=True)

# Make the price field useable for summary statistics and predictions   
    # Convert price to a float without £ symbol, and rename pcolunm name t state currency (£)
    # May convert to bins and even give categorical low, medium, high
products_na_removed[['price_gbp']] = products_na_removed[['price']].replace('[\£,]', '', regex=True).astype(float)
products_na_removed.drop('price', axis=1, inplace=True)

# Extract the various categories for products

    # Check which level provides a sensible number of categories by looking and NaN fields
        # Should be not too many or not too little and not too blank
# Find the max count of catgeories using '/' as the delimiter
max_categories = products_na_removed['category'].str.count('/').max()
# split by / and create multiple category fields
sub_categories = products_na_removed['category'].str.split('/', expand=True).reindex(range(max_categories), axis=1).add_prefix('sub_cat_')
# join the sub catgeories to the original dataframe and rename
products_na_removed = pd.concat([products_na_removed,sub_categories], axis=1)
# Remove original catgeory field
products_na_removed.drop('category', axis=1, inplace=True)
products_na_removed.head()

# #Remove stopwords, punctuation and numbers
# text2 = [remove_stopwords(x)\
#         .translate(str.maketrans('','',string.punctuation))\
#         .translate(str.maketrans('','',string.digits))\
#         for x in products_na_removed[['product_description']]]
# print(text2)






